## Phase 2: Preprocessing and Dataset Preparation

**Objective**: Prepare the Bangla Sentiment Dataset (columns: Tense, Label) for model training, ensuring compatibility with traditional (TF-IDF-based) and neural (BanglaBERT) sentiment classification models while preserving class imbalance characteristics.



### Step 1: Clean Text (Remove Noise, Normalize Bangla Script)

- **Objective**: Clean the `Tense` column to remove noise (e.g., special characters, URLs) and normalize Bangla text for consistency.

In [1]:
import pandas as pd
import re
from bnlp import CleanText
    
# Initialize BNLP cleaner
clean_text = CleanText(
    fix_unicode=True,
    unicode_norm=True,
    unicode_norm_form="NFKC",
    remove_url=False,
    remove_email=False,
    remove_emoji=False,
    remove_number=False,
    remove_digits=False,
    remove_punct=False,
    replace_with_url="<URL>",
    replace_with_email="<EMAIL>",
    replace_with_number="<NUMBER>",
    replace_with_digit="<DIGIT>",
    replace_with_punct = "<PUNC>"
)

punkt not found. downloading...


[nltk_data] Downloading package punkt_tab to /home/fahad/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
# Load the dataset from phase 1
dataset_path = "data-source/cleaned_dataset.csv"
df = pd.read_csv(dataset_path, encoding="utf-8")

df.head(3)

,Tense,Label
0,জিনিসপত্রের অতিরিক্ত দাম বৃদ্ধির জন্য এই শহরে ...,0
1,সঠিক ভাবে তদারকি করলে এই সমস্যা থেকে পরিত্রান ...,1
2,দেশের টাকা যখন বিদেশে চোলে যাচ্ছে তখন দেশের সর...,0


In [3]:
# Clean text using BNLP
def preprocess_text(text):
    # BNLP cleaning
    cleaned = clean_text(text)
    # Additional regex for URLs/hashtags (if BNLP misses any)
    cleaned = re.sub(r'http\S+|#\S+', '', cleaned)
    # Remove extra spaces
    cleaned = ' '.join(cleaned.split())
    return cleaned

df['Tense_Cleaned'] = df['Tense'].apply(preprocess_text)
    
# Check sample
print("Sample Cleaned Text:")
print(df[['Tense', 'Tense_Cleaned']].head(5))

Sample Cleaned Text:
                                               Tense  \
0  জিনিসপত্রের অতিরিক্ত দাম বৃদ্ধির জন্য এই শহরে ...   
1  সঠিক ভাবে তদারকি করলে এই সমস্যা থেকে পরিত্রান ...   
2  দেশের টাকা যখন বিদেশে চোলে যাচ্ছে তখন দেশের সর...   
3          ওনার মতো ব্যর্থ মন্ত্রীর পদত্যাগ করা উচিত   
4                 আল্লাহ তোদের বিচার করবে অপেক্ষা কর   

                                       Tense_Cleaned  
0  জিনিসপত্রের অতিরিক্ত দাম বৃদ্ধির জন্য এই শহরে ...  
1  সঠিক ভাবে তদারকি করলে এই সমস্যা থেকে পরিত্রান ...  
2  দেশের টাকা যখন বিদেশে চোলে যাচ্ছে তখন দেশের সর...  
3          ওনার মতো ব্যর্থ মন্ত্রীর পদত্যাগ করা উচিত  
4                 আল্লাহ তোদের বিচার করবে অপেক্ষা কর  


In [4]:
# Save the cleaned dataset
df.to_csv("outputs/cleaned_dataset.csv", encoding='utf-8', index=False)

In [6]:
# Check for null values in 'Tense_Cleaned' column
null_count = df['Tense_Cleaned'].isnull().sum()
print(f"Total null values in 'Tense_Cleaned': {null_count}")


Total null values in 'Tense_Cleaned': 0


### Step 2: Tokenize Texts for Traditional (TF-IDF) and Neural Models (BanglaBERT-Compatible Tokens)

- **Objective**: Tokenize cleaned text for traditional models (TF-IDF vectors) and neural models (BanglaBERT tokens).

In [7]:
# Import necessary libraries
from bnlp import NLTKTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoTokenizer
import numpy as np

In [10]:
# Initialize BNLP tokenizer
bnlp_tokenizer = NLTKTokenizer()

# TF-IDF Tokenization    
def bnlp_tokenize(text):
    return [token.strip().lower() for token in bnlp_tokenizer.word_tokenize(text)]

In [11]:
# text representation using tf-idf
tfidf_vectorizer = TfidfVectorizer(tokenizer=bnlp_tokenize, max_features=5000)  

tfidf_matrix = tfidf_vectorizer.fit_transform(df['Tense_Cleaned'])
    

In [14]:
# Validate tokenization
print("Sample TF-IDF Tokens:", bnlp_tokenizer.word_tokenize(df['Tense_Cleaned'].iloc[1])[:10])

Sample TF-IDF Tokens: ['সঠিক', 'ভাবে', 'তদারকি', 'করলে', 'এই', 'সমস্যা', 'থেকে', 'পরিত্রান', 'পওয়া', 'সম্ভব']


In [15]:
# Save TF-IDF matrix (sparse format)
np.savez("text_representation/tfidf_matrix.npz", 
         data=tfidf_matrix.data, 
         indices=tfidf_matrix.indices,
         indptr=tfidf_matrix.indptr, 
         shape=tfidf_matrix.shape
        )
 

In [16]:
# BanglaBERT Tokenization
tokenizer = AutoTokenizer.from_pretrained("sagorsarker/bangla-bert-base")

def tokenize_for_bert(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='np')

config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

In [17]:
# Tokenize in batches to manage memory
batch_size = 1000
bert_tokens = []
for i in range(0, len(df), batch_size):
    batch = df['Tense_Cleaned'][i:i+batch_size].tolist()
    tokens = tokenizer(batch, padding='max_length', truncation=True, max_length=128, return_tensors='np')
    bert_tokens.append(tokens['input_ids'])
    bert_input_ids = np.vstack(bert_tokens)

In [18]:
# Validate tokenization
print("Sample BERT Tokens:", tokenizer.convert_ids_to_tokens(bert_input_ids[0][:10]))

Sample BERT Tokens: ['[CLS]', 'জিনিস', '##পত', '##রে', '##র', 'অতি', '##রিক', '##ত', 'দাম', 'বদ']


In [19]:
# Save BERT tokens
np.save("text_representation/bert_input_ids.npy", bert_input_ids)


### Step 3: Split Dataset into Training, Validation, and Test Sets (Stratified)

- **Objective**: Split the dataset into training (80%), validation (10%), and test (10%) sets, preserving class imbalance.